In [1]:
import findspark
findspark.init("/opt/manual/spark/")

In [2]:
from pyspark.sql import SparkSession, functions as F

In [3]:
# Note the master is yarn and enableHiveSupport()

spark = SparkSession.builder \
.appName("When Otherwise") \
.master("local[2]") \
.getOrCreate()

2022-08-27 20:12:01,779 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
data =[("James","","Smith","36636","M",60000),
        ("Michael","Rose","","40288","M",70000),
        ("Robert","","Williams","42114","",400000),
        ("Maria","Anne","Jones","39192","F",500000),
        ("Jen","Mary","Brown","","F",0)]

cols = ["first_name","middle_name","last_name","dob","gender","salary"]

In [5]:
df = spark.createDataFrame(data, cols)

In [6]:
df.toPandas()

,first_name,middle_name,last_name,dob,gender,salary
0,James,,Smith,36636,M,60000
1,Michael,Rose,,40288,M,70000
2,Robert,,Williams,42114,,400000
3,Maria,Anne,Jones,39192,F,500000
4,Jen,Mary,Brown,,F,0


In [7]:
df2 = df.withColumn("new_gender", F.when(F.col("gender") == "M","Male") \
                            .when(F.col("gender") == "F","Female") \
                            .otherwise("Unknown"))

In [8]:
df2.toPandas()

,first_name,middle_name,last_name,dob,gender,salary,new_gender
0,James,,Smith,36636,M,60000,Male
1,Michael,Rose,,40288,M,70000,Male
2,Robert,,Williams,42114,,400000,Unknown
3,Maria,Anne,Jones,39192,F,500000,Female
4,Jen,Mary,Brown,,F,0,Female


In [9]:
df.createOrReplaceTempView("salary_temp")

In [10]:
spark.sql("""

SELECT CASE WHEN gender = 'M' THEN 'Male'
WHEN gender = 'F' THEN 'Female'
ELSE 'Unknown' END AS new_gender, first_name, gender
FROM salary_temp


""").toPandas()

,new_gender,first_name,gender
0,Male,James,M
1,Male,Michael,M
2,Unknown,Robert,
3,Female,Maria,F
4,Female,Jen,F


In [11]:
spark.stop()